In [1]:
# imports
from mavsdk import System
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time
import asyncio
import nest_asyncio
import numpy as np
from mediapipe.framework.formats import landmark_pb2
from threading import Thread
import tensorflow as tf
import math
import pandas as pd

2025-02-20 19:23:32.847082: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-20 19:23:32.849054: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-20 19:23:32.857702: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-20 19:23:32.883539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740108212.907871  169466 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740108212.91

In [2]:
# Initial Drone Attributes. Set initial/default parameters. These parameters are for hover
velocity = None
roll_default = 0.0
pitch_default = 0.0
throttle_default = 0.5
yaw_default = 0.0
# drone MC does not start initialized
drone_manual_control_initialized = False

# Initialize details on defaults
roll = roll_default
pitch = pitch_default
throttle = throttle_default
yaw = yaw_default

# Set video feed min requirements and model path
model_path = '/home/ryanli/PycharmProjects/UAV-Gesture-Recognization/Google-Gesture-Recognition/gesture_recognizer.task'
scale = 1.0 # less scale causes better (less laggy) video feed
fps = 30.0 # same with FPS I believe but it hasnt been tested

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Load tensorflow model
model = tf.keras.models.load_model('/home/ryanli/PycharmProjects/UAV-Gesture-Recognization/Google-Gesture-Recognition/GoogleCVTrained_LargerKernel.keras')
# Show the model architecture
print(model.summary())

# Create Names
model_name = ['dislike', 'fist', 'four', 'like', 'none', 'one', 'palm', 'peace', 'peace_inverted', 'rock', 'stop', 'three', 'three2', 'two_up', 'two_up_inverted']
threshold = 0.50
df_columns = ['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1', 'x_2', 'y_2', 'z_2', 'x_3', 'y_3', 'z_3', 'x_4', 'y_4', 'z_4', 'x_5', 'y_5', 'z_5', 'x_6', 'y_6', 'z_6', 'x_7', 'y_7', 'z_7', 'x_8', 'y_8', 'z_8', 'x_9', 'y_9', 'z_9', 'x_10', 'y_10', 'z_10', 'x_11', 'y_11', 'z_11', 'x_12', 'y_12', 'z_12', 'x_13', 'y_13', 'z_13', 'x_14', 'y_14', 'z_14', 'x_15', 'y_15', 'z_15', 'x_16', 'y_16', 'z_16', 'x_17', 'y_17', 'z_17', 'x_18', 'y_18', 'z_18', 'x_19', 'y_19', 'z_19', 'x_20', 'y_20', 'z_20']

2025-02-20 19:23:36.088299: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape (Reshape)               │ (None, 63, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 55, 32)         │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 27, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 19, 64)         │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 9, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 283,823 (1.08 MB)

 Trainable params: 94,607 (369.56 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 189,216 (739.13 KB)

None


In [3]:
# Creating Gesture Recognition model details
base_options = python.BaseOptions(model_asset_path=model_path)
VisionRunningMode = mp.tasks.vision.RunningMode
GestureRecognizerResult = mp.tasks.vision.GestureRecognizerResult
GestureRecognizer = mp.tasks.vision.GestureRecognizer
options = vision.GestureRecognizerOptions(
    base_options=base_options)
# For visualization, draw on the detected landmarks
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [4]:
# OpenCV details. Initialize videocapture camera on the zeroeth one, set width and height
cap = cv2.VideoCapture(0)
cap.set(3, int(256.0 * scale))
cap.set(4, int(144.0 * scale))
# make it run better
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))

True

In [5]:
# Helper Functions

# Change situations by command
def send_command(command):
    global roll, pitch, throttle, yaw
    try:
        if command == 'forward':
            print("moving forwards")
            roll = 0.0
            pitch = 0.5
            throttle = 0.5
            yaw = 0.0
        elif command == 'left':
            print("moving left")
            roll = -0.5
            pitch = 0.0
            throttle = 0.5
            yaw = 0.0
        elif command == 'backward':
            print("moving backwards")
            roll = 0.0
            pitch = -0.5
            throttle = 0.5
            yaw = 0.0
        elif command == 'right':
            print("moving right")
            roll = 0.5
            pitch = 0.0
            throttle = 0.5
            yaw = 0.0
        elif command == 'ascend':
            print("ascending")
            roll = 0.0
            pitch = 0.0
            throttle = 0.7
            yaw = 0.0
        elif command == 'descend':
            print("descending")
            roll = 0.0
            pitch = 0.0
            throttle = 0.2
            yaw = 0.0
        elif command == 'hover':
            print("hovering until further command")
            roll = roll_default
            pitch = pitch_default
            throttle = throttle_default
            yaw = yaw_default
        else:
            print("command '" + str(command) + "' received, reset speed")
            roll = roll_default
            pitch = pitch_default
            throttle = throttle_default
            yaw = yaw_default
    except:
        print("failed to recognize command: " + str(command))


# Parse gestures and map them to commands
def parse_gesture(gesture_object):
    global drone_manual_control_initialized
    result = gesture_object
    if result == "like":
        send_command("ascend")
    elif result == "dislike":
        send_command("descend")
    elif result == "fist":
        send_command("backward")
    elif result == "palm":
        send_command("forward")
    elif result == "one":
        send_command("left")
    elif result == "peace":
        send_command("right")
    elif result == "rock":
        # CHECK POINT DIRECTION. FOR NOW REMAIN HOVER
        send_command("hover")
    elif result == "four":
        print("Landing and Turning Off Drone")
        drone_manual_control_initialized = False
    else:
        # HOVER ON UNRECOGNIZED
        send_command("hover")

# Allow drone to always adhere to manual controls
async def manual_controls(drone):
    global roll, pitch, yaw, throttle
    while True:
        await drone.manual_control.set_manual_control_input(pitch, roll, throttle, yaw)

def flatten(gesture_recognition_result_landmarks):
    landmark_list = []
    for landmark in gesture_recognition_result_landmarks.hand_landmarks[0]:
        landmark_list.append([landmark.x, landmark.y, landmark.z])
    # normalize distance for distance between 5 and 6
    x_sq = math.pow(landmark_list[5][0] - landmark_list[6][0], 2)
    y_sq = math.pow(landmark_list[5][1] - landmark_list[6][1], 2)
    z_sq = math.pow(landmark_list[5][2] - landmark_list[6][2], 2)
    normalized_distance = math.sqrt(x_sq+y_sq+z_sq)
    # return landmarks to plain list
    plain_list = []
    # the most significant one is number 5. Normalize all values in relation to it
    for index in range(0, len(landmark_list)):
        for detail in range(0, 3):
            plain_list.append((landmark_list[index][detail] - landmark_list[5][detail]) / normalized_distance)
    df = pd.DataFrame(plain_list).T
    # return results
    return df

In [6]:
# Main thread. Parses connection of drone and manual controls to make stuff work
async def main():
    """Main function to connect to the drone and input manual controls"""
    global roll, pitch, yaw, throttle
    global drone_manual_control_initialized
    # Connect to the Simulation
    drone = System()
    await drone.connect(system_address="udp://:14540")

    # This waits till a mavlink based drone is connected
    print("Waiting for drone to connect...")
    async for state in drone.core.connection_state():
        if state.is_connected:
            print(f"-- Connected to drone!")
            break

    # Checking if Global Position Estimate is ok
    async for health in drone.telemetry.health():
        if health.is_global_position_ok and health.is_home_position_ok:
            print("-- Global position state is good enough for flying.")
            break

    # Arming the drone
    print("-- Arming")
    await drone.action.arm()

    # Take off
    print("-- Taking off")
    await drone.action.takeoff()
    await asyncio.sleep(10)

    print("-- Set manual control")
    roll = 0.0
    pitch = 0.0
    throttle = 0.5
    yaw = 0.0
    # set the manual control input after arming
    manual = asyncio.create_task(manual_controls(drone))
    print("-- wait")
    await asyncio.sleep(1)

    # start manual control
    print("-- Starting manual control")
    await drone.manual_control.start_position_control()
    # drone is now initialized
    drone_manual_control_initialized = True
    print("-- wait")
    await asyncio.sleep(1)
    print("-- Activating manual control")

    thread = Thread(target=recognizer_threaded)
    thread.start()

    while drone_manual_control_initialized:
        await asyncio.sleep(1)

    await drone.action.land()
    print("-- end")

# A threaded recognizer that activates commands by detecting gestures
def recognizer_threaded(testing = False):
    global drone_manual_control_initialized

    if testing:
        print("TESTING ON")

    with GestureRecognizer.create_from_options(options) as recognizer:
        while cap.isOpened() and drone_manual_control_initialized:
            success, image = cap.read()
            time.sleep(1.0/fps)
            if not success:
              print("Ignoring empty camera frame.")
              # If loading a video, use 'break' instead of 'continue'.
              continue

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
            recognition_result = recognizer.recognize(mp_image)

            # Recognizer Function
            try:
                hand_landmarks = recognition_result.hand_landmarks
                flattened_result = flatten(recognition_result)
                model_result =  model(flattened_result)
                max_index = np.argmax(model_result[0])
                top_gesture = model_name[max_index] if model_result[0][max_index] > threshold else "none"
                if testing:
                    print("Detected hand. Probably " + top_gesture + "?")
                else:
                    parse_gesture(top_gesture)
            except Exception as e:
                print("Failed to detect. Hand not present?")
                cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
                if cv2.waitKey(5) & 0xFF == 27:
                  break
                continue

            # Draw the hand annotations on the image.
            image.flags.writeable = True
            hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
            hand_landmarks_proto.landmark.extend([
                landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks[0]
            ])
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks_proto,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
            # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
            if cv2.waitKey(5) & 0xFF == 27:
              break
    if drone_manual_control_initialized:
        # something is wrong. terminalize and drop all things
        drone_manual_control_initialized = False
    cap.release()

In [7]:
# start
asyncio.run(main())

Waiting for drone to connect...
-- Connected to drone!
-- Global position state is good enough for flying.
-- Arming
-- Taking off
-- Set manual control
-- wait
-- Starting manual control
-- wait
-- Activating manual control


I0000 00:00:1740108256.642820  171703 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1740108256.661448  171721 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.3.4 - kisak-mesa PPA), renderer: llvmpipe (LLVM 17.0.6, 256 bits)
W0000 00:00:1740108256.662121  171703 gesture_recognizer_graph.cc:129] Hand Gesture Recognizer contains CPU only ops. Sets HandGestureRecognizerGraph acceleration to Xnnpack.
I0000 00:00:1740108256.663345  171703 hand_gesture_recognizer_graph.cc:250] Custom gesture classifier is not defined.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1740108256.709343  171722 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740108256.739849  171727 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740

Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not present?
Failed to detect. Hand not p

In [ ]:
asyncio.run(recognizer_threaded(testing=True))

In [8]:
cap.release()
loop = asyncio.get_running_loop()
loop.stop()